In [ ]:
import pandas as pd
import numpy as np

In [ ]:
loans = pd.read_csv("loans.csv")
emotions_20 = pd.read_csv("emotion_score_2020.csv")
emotions_old = pd.read_csv("emotion_score.csv")
field_partner = pd.read_csv("field_partner_info.csv")

In [ ]:
loans.groupby(['STATUS'])['STATUS'].count()

STATUS
expired          90586
fundRaising       5599
funded         1846044
refunded          8895
Name: STATUS, dtype: int64

In [ ]:
emotions = pd.concat([emotions, emotions_old], axis=0)

In [ ]:
emotions.drop_duplicates(subset=['LOAN_ID'], inplace=True)

In [ ]:
#converting to date_time_format
loans['POSTED_DATE'] = pd.to_datetime(loans['POSTED_TIME']).dt.date
loans['PLANNED_EXPIRATION_DATE'] = pd.to_datetime(loans['PLANNED_EXPIRATION_TIME']).dt.date

#filtering 2019 (train-val data) and 2020 loans (prediction horizon)
loans_1920 = loans[(loans['POSTED_DATE'].astype(str)>'2019-01-01') & (loans['POSTED_DATE'].astype(str)<'2020-04-01')]

#columns that need to be dopped
loan_lender_drops = ['LOAN_NAME', 'DESCRIPTION', 'ORIGINAL_LANGUAGE','IMAGE_ID', 'POSTED_TIME', 'VIDEO_ID', 'ACTIVITY_NAME', 'LOAN_USE', 'COUNTRY_CODE', 'TOWN_NAME', 'CURRENCY_POLICY', 'CURRENCY_EXCHANGE_COVERAGE_RATE', 'CURRENCY', 'DISBURSE_TIME', 'RAISED_TIME', 'NUM_BULK_ENTRIES', 'BORROWER_NAMES', 'REPAYMENT_INTERVAL']
loans_1920 = loans_1920.drop(columns=loan_lender_drops)

#columns that need to be dropped after feature engineering
drop_post_eng = ['DESCRIPTION_TRANSLATED', 'FUNDED_AMOUNT', 'FUNDED_RATIO', 'TAGS', 'DISTRIBUTION_MODEL', 'SECTOR_NAME', 'COUNTRY_NAME', 'PLANNED_EXPIRATION_DATE', 'PLANNED_EXPIRATION_TIME', 'BORROWER_GENDERS', 'DISTRIBUTION_MODEL']

#feature engineering
#create length of description
loans_1920['LEN_DESCRIPTION'] = loans_1920['DESCRIPTION_TRANSLATED'].astype(str).apply(lambda x: len(x))
#create funding ratio
loans_1920['FUNDED_RATIO'] = loans_1920['FUNDED_AMOUNT']/loans_1920['LOAN_AMOUNT']

#separate out underfunded and fully-funded loans
loans_1920_underfunded = loans_1920[(loans_1920['FUNDED_RATIO']<1)]
loans_1920_funded = loans_1920[loans_1920['FUNDED_RATIO']>=1]
loans_1920_model =  pd.concat([loans_1920_underfunded, loans_1920_funded])

#create length of tags
loans_1920_model['LEN_TAGS'] = loans_1920_model['TAGS'].astype(str).apply(lambda x: len(x.split()))
#re-initialize status as 1 for under-funded and 0 for fully funded
loans_1920_model['STATUS'] = np.where(loans_1920_model['STATUS'] == "expired", 1,0)

#create dummy variable for sector and country name
loans_1920_model = pd.concat([loans_1920_model, pd.DataFrame(pd.get_dummies(loans_1920_model['SECTOR_NAME']))], axis=1)
loans_1920_model = pd.concat([loans_2019_model, pd.DataFrame(pd.get_dummies(loans_1920_model['COUNTRY_NAME']))], axis=1)

#feature for days to expire
loans_1920_model['DAYS_TO_EXPIRE'] = (loans_1920_model['PLANNED_EXPIRATION_DATE'] - loans_1920_model['POSTED_DATE'])/np.timedelta64(1, 'D')

In [ ]:
#function to crcount number of borrowers from each gender
def gender_count(x,val):
  count = 0
  if val == 'male':
    for i in x:
      if i=='male':
        count = count + 1
  elif val == 'female':
    for i in x:
      if i=='female':
        count = count + 1
  return count

#create gender features
loans_1920_model['MALE_BORROWERS'] = loans_1920_model['BORROWER_GENDERS'].astype(str).apply(lambda x: gender_count((x.split(", ")), 'male'))
loans_1920_model['FEMALE_BORROWERS'] = loans_1920_model['BORROWER_GENDERS'].astype(str).apply(lambda x: gender_count((x.split(", ")), 'female'))

#function to count number of picutres of borrowers
def picture_count(x,val):
  count = 0
  if val == 'true':
    for i in x:
      if (i == 'true') or (i=='TRUE'):
        count = count + 1
  elif val == 'false':
    for i in x:
      if i=='false':
        count = count + 1
  return count

#create features for # borrower pictures  
loans_1920_model['BORROWER_PICTURED'] = np.where(loans_1920_model['BORROWER_PICTURED'].astype(str).apply(lambda x: picture_count((x.split()), 'true'))==1, 1,0)

#only keep those loans with field partners
loans_1920_model = loans_1920_model[loans_1920_model['DISTRIBUTION_MODEL']=='field_partner']

#drop columns after feature engineering
loans_1920_model_post_fe = loans_1920_model.drop(columns=drop_post_eng)

In [ ]:
#Dropping columns not required from field partner list
partner_drop = ['kiva_borrowers', 'currency_exchange_loss_rate', 'fundraising_status', 'country', 'capital', 'official_language', 'population', 'avg_annual_income', 'labour_force', 'population_below_poverty_line', 'literacy_rate', 'infant_mortality_rate_per_1000', 'life_expectancy']
drop_more = ['id', 'PARTNER_ID']
field_partner = field_partner.drop(columns=partner_drop)

#merging field partner info with loan list
loans_1920_model_post_fe = loans_1920_model_post_fe.merge(field_partner, left_on = 'PARTNER_ID', right_on='id', how='left')

#getting emotional scores for loan description
loans_1920_model_post_fe = loans_1920_model_post_fe.merge(emotions[['LOAN_ID', 'emo_score']], on='LOAN_ID', how='left')
loans_1920_model_post_fe = loans_1920_model_post_fe.drop(columns=drop_more)

In [ ]:
#create nan values - which can be later imputed with median values of that columns
loans_1920_model_post_fe['partner_rating'].replace(['Inactive'], np.nan, inplace=True)
loans_1920_model_post_fe['profitability_ROA'].replace([' N/A '], np.nan, inplace=True)
loans_1920_model_post_fe['avg_loan_size'].replace([' N/A '], np.nan, inplace=True)
loans_1920_model_post_fe['avg_cost_to_borrower'].replace([' N/A '], np.nan, inplace=True)

In [ ]:
#removing special chars
chars_to_remove = ['$', ',', ' months', ' deaths', '\t', '\n', '%', 'APR', 'PY',')', '(','  See note']
missing_val_col = ['delinquency_rate', 'loans_at_risk_rate', 'time_on_kiva', 'total_loans', 'avg_cost_to_borrower', 'profitability_ROA', 'avg_loan_size', 'default_rate']
for i in missing_val_col:
  for j in chars_to_remove:
    loans_1920_model_post_fe[i] = loans_2019_model_post_fe[i].str.replace(j, '')

In [ ]:
#creating binary feature for interest fees charged
loans_1920_model_post_fe['are_interest_fees_charged'] = np.where(loans_1920_model_post_fe['are_interest_fees_charged']=='Yes', 1,0)

In [ ]:
#imputing null values with median values
null_cols = ['time_on_kiva', 'avg_loan_size', 'emo_score', 'partner_rating', 'profitability_ROA', 'avg_cost_to_borrower', 'total_loans', 'delinquency_rate', 'loans_at_risk_rate', 'default_rate']
for i in null_cols:
    loans_1920_model_post_fe[i] = loans_1920_model_post_fe[i].fillna((loans_1920_model_post_fe[i].astype(float).median()))

In [ ]:
#check if there are any more nan values
pd.DataFrame(loans_2019_model_post_fe.isnull().sum()).sort_values(by=0, ascending=False).head(13)

,0
LOAN_ID,0
Tajikistan,0
Sierra Leone,0
Senegal,0
Samoa,0
Rwanda,0
Puerto Rico,0
Philippines,0
Peru,0
Paraguay,0


In [ ]:
#filtering out 2019 values - train+val
df_2019 = loans_1920_model_post_fe[(loans_1920_model_post_fe['POSTED_DATE'].astype(str)<'2020-01-01')]

#saving as csv
df_2019.to_csv('train_2019.csv')

#filtering out 2020 values - prediction horizon
df_2020_q1 = loans_1920_model_post_fe[(loans_1920_model_post_fe['POSTED_DATE'].astype(str)>'2019-12-31') & (loans_1920_model_post_fe['POSTED_DATE'].astype(str)<'2020-04-01')]
df_2020_q1.drop_duplicates(subset=['LOAN_ID'], inplace=True)
#saving as csv
df_2020_q1.to_csv('test_2020.csv')